## Formulação Padrão (ou Canônica) em Programação Linear

A **formulação padrão** de um problema de Programação Linear (PL) consiste em escrever o problema da seguinte forma:

### Estrutura da Formulação Padrão:

$$
\begin{aligned}
\text{Minimizar } & \quad c^T x \\
\text{sujeito a } & \quad Ax = b \\
                  & \quad x \geq 0
\end{aligned}
$$

Onde:
- \( x \): vetor de variáveis de decisão  
- \( c \): vetor de coeficientes da função objetivo  
- \( A \): matriz de coeficientes das restrições  
- \( b \): vetor de constantes das restrições  

---

### Por que utilizar a formulação padrão?

1. ✅ **Compatibilidade com algoritmos clássicos**  
   Muitos algoritmos de solução (como o Método Simplex e métodos de ponto interior) requerem que o problema esteja nessa forma para funcionarem corretamente.

2. ✅ **Uniformidade matemática**  
   Permite representar qualquer problema de PL de maneira padronizada, facilitando a análise teórica (dualidade, condições KKT, etc).

3. ✅ **Transformação de desigualdades**  
   Restrições do tipo:

   $$
   a^T x \leq b
   $$

   Podem ser convertidas para igualdades com **variáveis de folga**:

   $$
   a^T x + s = b \quad \text{com } s \geq 0
   $$

   Isso permite manter o problema no formato exigido.

4. ✅ **Facilita interpretação econômica**  
   As variáveis de folga e variáveis duais (multiplicadores de Lagrange) têm interpretações claras como:  
   - Recursos excedentes  
   - Preço sombra de recursos escassos  

5. ✅ **Requisito interno de solvers**  
   Solvers como `cvxopt`, `scipy.optimize`, `Gurobi`, entre outros, convertem internamente os problemas para essa forma padrão.

---

### Exemplo de conversão de uma restrição:

Restrição original:
$$
2x_1 + 3x_2 \leq 10
$$

Forma padrão (introduzindo variável de folga \( s_1 \geq 0 \)):
$$
2x_1 + 3x_2 + s_1 = 10
$$

---

### Observação:
Caso o problema original seja de **maximização**, basta multiplicar a função objetivo por \( -1 \) para transformá-lo em um problema de **minimização** (formato padrão).


## Problema Maximização do Lucro no Cultivo de Cereais

### Conversão para a Formulação Padrão

#### Variáveis de decisão:
- \( x \): área (em ares) destinada ao cereal A  
- \( y \): área (em ares) destinada ao cereal B  

---

#### Função Objetivo (Lucro)

- Cereal A:  
  - Produz 8 sacas por are  
  - Preço de venda: R\$ 150/saca  
  - Lucro por are: \( 8 * 150 = 1200 \)  
  - Custo de mão de obra: \( 3 * 200 = 600 \)  
  - **Lucro líquido por are de A**: \( 1200 - 600 = 600 \)

- Cereal B:  
  - Produz 10 sacas por are  
  - Preço de venda: R\$ 120/saca  
  - Lucro por are: \( 10 * 120 = 1200 \)  
  - Custo de mão de obra: \( 2 * 200 = 400 \)  
  - **Lucro líquido por are de B**: \( 1200 - 400 = 800 \)

A função objetivo é:

$$
\text{Maximizar } Z = 600x + 800y
$$

Para a formulação padrão (minimização), multiplicamos por \(-1\):

$$
\text{Minimizar } -600x -800y
$$

---

#### Restrições

1. **Limite de área total disponível (100 ares):**

$$
x + y \leq 100 \quad \Rightarrow \quad x + y + s_1 = 100, \quad s_1 \geq 0
$$

2. **Limite de mão de obra (240 homens-hora):**

$$
3x + 2y \leq 240 \quad \Rightarrow \quad 3x + 2y + s_2 = 240, \quad s_2 \geq 0
$$

3. **Demanda máxima do cereal A (480 sacas = 60 ares):**

$$
x \leq 60 \quad \Rightarrow \quad x + s_3 = 60, \quad s_3 \geq 0
$$

4. **Demanda máxima do cereal B (800 sacas = 80 ares):**

$$
y \leq 80 \quad \Rightarrow \quad y + s_4 = 80, \quad s_4 \geq 0
$$

---

#### Condições de não negatividade

$$
x \geq 0, \quad y \geq 0, \quad s_1 \geq 0, \quad s_2 \geq 0, \quad s_3 \geq 0, \quad s_4 \geq 0
$$

---

#### Resumo

A formulação padrão completa do problema é:

$$
\begin{aligned}
\text{Minimizar } \quad & -600x -800y \\
\text{sujeito a} \quad
& x + y + s_1 = 100 \\
& 3x + 2y + s_2 = 240 \\
& x + s_3 = 60 \\
& y + s_4 = 80 \\
& x, y, s_1, s_2, s_3, s_4 \geq 0
\end{aligned}
$$


In [4]:
from cvxopt.modeling import variable, op
from cvxopt import solvers

# Variáveis de decisão
x = variable()
y = variable()

# Variáveis de folga
s1 = variable()
s2 = variable()
s3 = variable()
s4 = variable()

# Função objetivo (minimizar o negativo do lucro)
lucro = -600 * x - 800 * y

# Restrições como equações com folgas
restricoes = [
    x + y + s1 == 100,     # Área total
    3 * x + 2 * y + s2 == 240,  # Mão de obra
    x + s3 == 60,          # Demanda cereal A
    y + s4 == 80,          # Demanda cereal B
    x >= 0, y >= 0,        # Não negatividade
    s1 >= 0, s2 >= 0, s3 >= 0, s4 >= 0
]

# Modelo de otimização
modelo = op(lucro, restricoes)
modelo.solve('dense', 'glpk')

# Resultados
print("Status:", modelo.status)
print("Área de cultivo para cereal A (x):", x.value[0])
print("Área de cultivo para cereal B (y):", y.value[0])
print("Lucro máximo: R$", -modelo.objective.value()[0])
print("Variável dual associada a restrição de área R$", restricoes[0].multiplier.value[0])
print("Variável dual associada a restrição de mão de obra R$", restricoes[1].multiplier.value[0])
print("Variável dual associada a demanda máxima do cereal A R$", restricoes[2].multiplier.value[0])
print("Variável dual associada a demanda máxima do cereal B R$", restricoes[3].multiplier.value[0])
print("Valor da variável de folga S1:", s1.value[0])
print("Valor da variável de folga S2:", s2.value[0])
print("Valor da variável de folga S3:", s3.value[0])
print("Valor da variável de folga S4:", s4.value[0])


Status: optimal
Área de cultivo para cereal A (x): 20.0
Área de cultivo para cereal B (y): 80.0
Lucro máximo: R$ 76000.0
Variável dual associada a restrição de área R$ 600.0
Variável dual associada a restrição de mão de obra R$ 0.0
Variável dual associada a demanda máxima do cereal A R$ 0.0
Variável dual associada a demanda máxima do cereal B R$ 200.0
Valor da variável de folga S1: 0.0
Valor da variável de folga S2: 20.0
Valor da variável de folga S3: 40.0
Valor da variável de folga S4: 0.0


Como o ponto ótimo não está sobre as **restrições de mão de obra** e **demanda do cereal A**, as respectivas variáveis de folga **S2** e **S3** retornam um valor maior que 0 (Restrição não ativa), enquanto que as curvas de **restrições de área total** e **demanda do cereal B** estão no ponto ótimo e suasl folgas **S1** e **S4** retornam valor igual a zero (Restrição ativa).

## Utilizando as classes **solvers** e **matrix** da biblioteca **CVXOPT**

In [13]:
from cvxopt import matrix, solvers

A = matrix([ [1., 3., 1., 0.], [1., 2., 0., 1.]])
b = matrix([ [100., 240., 60., 80.]])
c = matrix([-600., -800.])
sol=solvers.lp(c, A, b)
print(sol['x'])

     pcost       dcost       gap    pres   dres   k/t
 0: -7.5059e+04 -1.3201e+05  1e+04  5e-02  8e-01  1e+00
 1: -7.3485e+04 -9.8633e+04  5e+03  2e-02  3e-01  2e+02
 2: -7.6892e+04 -1.0286e+05  6e+03  2e-02  4e-01  5e+02
 3: -7.6009e+04 -7.7384e+04  2e+02  1e-03  2e-02  9e+01
 4: -7.6000e+04 -7.6014e+04  2e+00  1e-05  2e-04  9e-01
 5: -7.6000e+04 -7.6000e+04  2e-02  1e-07  2e-06  9e-03
 6: -7.6000e+04 -7.6000e+04  2e-04  1e-09  2e-08  9e-05
Optimal solution found.
[ 2.00e+01]
[ 8.00e+01]



### Forma Padrão

In [17]:
from cvxopt import matrix, solvers
import numpy as np

c = matrix([-600., -800., 0., 0., 0., 0.])
Aeq = matrix([[1., 3., 1., 0.],  # x1
             [1., 2., 0., 1.],  # x2
             [1., 0., 0., 0.],  # s1
             [0., 1., 0., 0.],  # s2
             [0., 0., 1., 0.],  # s3
             [0., 0., 0., 1.]]  # s4
             )
beq = matrix([100.0, 240.0, 60.0, 80.0])
G = matrix(-np.eye(6))
h = matrix(np.zeros(6))

print(G)

sol = solvers.lp(c, G, h, Aeq, beq)
print(sol['x'])

[-1.00e+00 -0.00e+00 -0.00e+00 -0.00e+00 -0.00e+00 -0.00e+00]
[-0.00e+00 -1.00e+00 -0.00e+00 -0.00e+00 -0.00e+00 -0.00e+00]
[-0.00e+00 -0.00e+00 -1.00e+00 -0.00e+00 -0.00e+00 -0.00e+00]
[-0.00e+00 -0.00e+00 -0.00e+00 -1.00e+00 -0.00e+00 -0.00e+00]
[-0.00e+00 -0.00e+00 -0.00e+00 -0.00e+00 -1.00e+00 -0.00e+00]
[-0.00e+00 -0.00e+00 -0.00e+00 -0.00e+00 -0.00e+00 -1.00e+00]

     pcost       dcost       gap    pres   dres   k/t
 0: -6.6629e+04 -6.6629e+04  2e+04  3e-17  4e-01  1e+00
 1: -7.3289e+04 -7.3086e+04  1e+04  1e-14  2e-01  2e+02
 2: -7.4529e+04 -7.4380e+04  3e+03  2e-14  5e-02  1e+02
 3: -7.5959e+04 -7.5936e+04  3e+02  2e-14  5e-03  2e+01
 4: -7.6000e+04 -7.5999e+04  3e+00  2e-14  5e-05  2e-01
 5: -7.6000e+04 -7.6000e+04  3e-02  3e-14  5e-07  2e-03
 6: -7.6000e+04 -7.6000e+04  3e-04  2e-14  5e-09  2e-05
Optimal solution found.
[ 2.00e+01]
[ 8.00e+01]
[ 4.76e-08]
[ 2.00e+01]
[ 4.00e+01]
[ 7.44e-08]



---